## IMPORTS


In [2]:
import re
import os
from tika import parser
import spacy
from spacy_langdetect import LanguageDetector
import pytesseract
from PIL import Image
import codecs
import sys
import json
from wand.image import Image as Img

## Check for scans in txt files


In [6]:
YEAR = 2017
# Get the list of all suspected scans
suspected_scans = []

with open(f'../assets/json/suspected_scans/suspected_scans_{YEAR}.json', 'r') as json_file:
    suspected_scans = json.load(json_file)
    
    
for unique_file in suspected_scans:
    path_pdf = f'../assets/pdf/{YEAR}/pdf-{unique_file}.pdf'
    path_txt = f'../assets/txt/{YEAR}/txt-{unique_file}.txt'
    path_img = f'../assets/img/img{unique_file}.jpg'
    path_dir_img = '../assets/img/'
    
    
    # Convert pdf to jpg
    with Img(filename=path_pdf, resolution=300) as img:
        img.compression_quality = 99
        img.save(filename=path_img)
    # Apply OCR on it
    current_file = ''
    for r, d, f in os.walk(path_dir_img):
        for img in f:
            if '.jpg' in img:
                print(f'{img} Converting...')
                text_file = pytesseract.image_to_string(Image.open(path_dir_img + img))
                current_file += text_file
                os.remove(path_dir_img + img)
                print(f'{img} Deleted')
                
    with codecs.open(path_txt, 'w', 'utf-8') as file:
        file.write(current_file)
    
    print(f'Scan N° {unique_file} as been converted as txt at: {path_txt}')
    print('--------------------------------------------------')